### Intoduction to Polars

https://realpython.com/python-gil/

https://pola.rs/

![alt text](img/polars.png)

The versatility of Python and its simple syntax are certainly the strong points of this high-level, general-purpose programming language. However, one of its greatest weaknesses, if not the greatest (especially considering how the computing ecosystem has evolved), is the Global Interpreter Lock (GIL).

The Python Global Interpreter Lock, or GIL, is a mutex (lock) that permits only one thread to control the Python interpreter at a time. In simple terms, it means that only one thread can be actively executing code at any given moment. While this constraint may not be noticeable in single-threaded programs, it can become a performance bottleneck in scenarios involving multi-threaded code or CPU-bound tasks.

The GIL's reputation as an "infamous" feature stems from its restriction of executing only one thread at a time, even in multi-threaded architectures with multiple CPU cores. This article explores how the GIL affects the performance of Python programs and provides insights into mitigating its impact on code.

To understand the problem the GIL addresses, it's crucial to delve into Python's memory management using reference counting. Objects in Python have a reference count variable that tracks the number of references pointing to the object. When this count reaches zero, the object's memory is released. The GIL addresses the challenge of protecting this reference count variable from race conditions where two threads might simultaneously increase or decrease its value. Without proper protection, such scenarios could lead to memory leaks or, worse, incorrect release of memory while references to the object still exist, resulting in crashes or unpredictable bugs.

In [1]:
import sys
a = []
b = a
sys.getrefcount(a)

3

While one solution could be adding locks to shared data structures to safeguard the reference count variable, it introduces the risk of deadlocks and performance degradation due to repeated lock acquisition and release. The GIL takes a different approach by acting as a single lock on the interpreter itself. This ensures that executing any Python bytecode necessitates acquiring the interpreter lock. Although this approach avoids deadlocks and minimizes performance overhead, it effectively confines CPU-bound Python programs to single-threaded execution.

Python is a high-level programming language that prioritizes ease of use and readability. However, this focus on simplicity and readability can impact its performance. Recognizing the need for optimized execution in certain scenarios, an interface between Python and C has been developed. This integration allows developers to leverage the efficiency of C, a low-level language known for its speed, in performance-critical sections of their Python programs. By doing so, they can strike a balance between Python's simplicity and C's performance, optimizing their applications for specific tasks. Interestingly, the GIL was not only designed to ensure better performance in single-threaded programs but also to facilitate the integration of C libraries that were not thread-safe. It's amusing to note that in C, you can sidestep the issue of individual threads, and this playful workaround extends to other programming languages as well.

Those clever creators of Polars wrote the library in Rust. In addition to this, they focused on parallelization and efficiency. And given the excellent result, Polars will probably replace Pandas in the course of a few years. Therefore, in this course, we explore a bit of this fantastic library.

In [2]:
try:
    import polars as pl
except ImportError:
    print("Il pacchetto 'polars' non è installato. Installazione in corso...")
    %conda install -c conda-forge polars -y
    print("Installazione completata.")


### Data types

Polars is entirely based on Arrow data types and backed by Arrow memory arrays. From this point of view, there isn't much new, but it's worth listing the types we will use the most:

* ```pl.Int32``` and ```pl.Int64```
* ```pl.Float32``` and ```pl.Float32```
* ```pl.Date``` and ```pl.Datetime```
* ```pl.Boolean``` and ```pl.Categorical```

Categorical data represents string data where the values in the column have a finite set of values. Storing these values as plain strings is a waste of memory so Polars encode them in dictionary format.

Keep in mind that in Polars, 'NaN' doesn't exist; instead, it is replaced with ```pl.Null```.

### Data structures

Regarding the structures, they are similar to those in Pandas, so essentially, we will be working with:
* ```pl.Series``` and  ```pl.DataFrame```

Some of these functions have been implemented and operate for ```pl.DataFrame``` in the same way as those in Pandas:

* ```.head()``` shows the first 5 elements
* ```.tail()``` shows the last 5 elements
* ```.sample()``` shows 5 random elements
* ```.describe()``` returns summary statistics

### I/O

Polars supports reading and writing to all common files (e.g. csv, json, parquet), cloud storage (S3, Azure Blob, BigQuery) and databases (e.g. postgres, mysql). For this course, we are revisiting the old and dear Titanic database:

In [3]:
df = pl.read_csv("titanic.csv")

In general, the syntax is quite simple: use ```read_filetype``` for reading and ```write_filetype``` for writing. I would refer you to the documentation for the attributes of the aforementioned functions. A non-exhaustive list of file types includes:
* ```.read_json()``` and ```.write_json()```
* ```.read_parquet()``` and ```.write_parquet()```

### Contexts

Contexts and Expressions constitute the language through which Polars performs operations on data. A context pertains to the circumstances under which an expression is meant to be assessed. Let's do some examples:

##### Select

In the ```select``` context the selection applies expressions over columns. The expressions in this context must produce Series that are all the same length:

In [4]:
out = df.select(pl.col("Name"),
                pl.col("Age"),
                pl.col("Survived")
                ).limit(5)
print(out)

shape: (5, 3)
┌───────────────────────────────────┬──────┬──────────┐
│ Name                              ┆ Age  ┆ Survived │
│ ---                               ┆ ---  ┆ ---      │
│ str                               ┆ f64  ┆ i64      │
╞═══════════════════════════════════╪══════╪══════════╡
│ Braund, Mr. Owen Harris           ┆ 22.0 ┆ 0        │
│ Cumings, Mrs. John Bradley (Flor… ┆ 38.0 ┆ 1        │
│ Heikkinen, Miss. Laina            ┆ 26.0 ┆ 1        │
│ Futrelle, Mrs. Jacques Heath (Li… ┆ 35.0 ┆ 1        │
│ Allen, Mr. William Henry          ┆ 35.0 ┆ 0        │
└───────────────────────────────────┴──────┴──────────┘


```select``` is very similar to ```with_columns``` The main difference is that the latter retains the original columns and adds new ones while ```select``` drops the original columns:

In [5]:
out = out.with_columns(pl.col("Survived").cast(pl.Boolean).alias("As bool"))
print(out)

shape: (5, 4)
┌───────────────────────────────────┬──────┬──────────┬─────────┐
│ Name                              ┆ Age  ┆ Survived ┆ As bool │
│ ---                               ┆ ---  ┆ ---      ┆ ---     │
│ str                               ┆ f64  ┆ i64      ┆ bool    │
╞═══════════════════════════════════╪══════╪══════════╪═════════╡
│ Braund, Mr. Owen Harris           ┆ 22.0 ┆ 0        ┆ false   │
│ Cumings, Mrs. John Bradley (Flor… ┆ 38.0 ┆ 1        ┆ true    │
│ Heikkinen, Miss. Laina            ┆ 26.0 ┆ 1        ┆ true    │
│ Futrelle, Mrs. Jacques Heath (Li… ┆ 35.0 ┆ 1        ┆ true    │
│ Allen, Mr. William Henry          ┆ 35.0 ┆ 0        ┆ false   │
└───────────────────────────────────┴──────┴──────────┴─────────┘


##### Filter

In the filter context you filter the existing dataframe based on arbitrary expression:

In [6]:
out = df.filter((pl.col("Age") < 10) & (pl.col("Pclass") == 3)).limit(5)
print(out)

shape: (5, 12)
┌─────────────┬──────────┬────────┬──────────────────┬───┬─────────┬─────────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name             ┆ … ┆ Ticket  ┆ Fare    ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---              ┆   ┆ ---     ┆ ---     ┆ ---   ┆ ---      │
│ i64         ┆ i64      ┆ i64    ┆ str              ┆   ┆ str     ┆ f64     ┆ str   ┆ str      │
╞═════════════╪══════════╪════════╪══════════════════╪═══╪═════════╪═════════╪═══════╪══════════╡
│ 8           ┆ 0        ┆ 3      ┆ Palsson, Master. ┆ … ┆ 349909  ┆ 21.075  ┆ null  ┆ S        │
│             ┆          ┆        ┆ Gosta Leonard    ┆   ┆         ┆         ┆       ┆          │
│ 11          ┆ 1        ┆ 3      ┆ Sandstrom, Miss. ┆ … ┆ PP 9549 ┆ 16.7    ┆ G6    ┆ S        │
│             ┆          ┆        ┆ Marguerite Rut   ┆   ┆         ┆         ┆       ┆          │
│ 17          ┆ 0        ┆ 3      ┆ Rice, Master.    ┆ … ┆ 382652  ┆ 29.125  ┆ null  ┆ Q        │
│    

##### Group by

Actually the ```group_by``` context create a new DataFrame that includes the different 'groups' we want to group together

In [7]:
df.group_by('Sex')

this gives an expression, in order to obtain a Dataframe or a Series we have to evaluate it

In [8]:
out = df.group_by('Sex', maintain_order=True).count()
print(out)

shape: (2, 2)
┌────────┬───────┐
│ Sex    ┆ count │
│ ---    ┆ ---   │
│ str    ┆ u32   │
╞════════╪═══════╡
│ male   ┆ 577   │
│ female ┆ 314   │
└────────┴───────┘


In the ```group_by``` context, expressions work on groups and thus can yield results of any length:

In [9]:
out = df.group_by("Pclass").agg(
    pl.count("PassengerId").alias("#"),
    pl.col("PassengerId").filter(pl.col("Name").str.contains("John")).alias("ids of Johns"),
    pl.sum("Survived")
)
print(out)

shape: (3, 4)
┌────────┬─────┬─────────────────┬──────────┐
│ Pclass ┆ #   ┆ ids of Johns    ┆ Survived │
│ ---    ┆ --- ┆ ---             ┆ ---      │
│ i64    ┆ u32 ┆ list[i64]       ┆ i64      │
╞════════╪═════╪═════════════════╪══════════╡
│ 1      ┆ 216 ┆ [2, 169, … 823] ┆ 136      │
│ 2      ┆ 184 ┆ [42, 99, … 865] ┆ 87       │
│ 3      ┆ 491 ┆ [9, 46, … 889]  ┆ 119      │
└────────┴─────┴─────────────────┴──────────┘


##### Sorting

It is often useful to be able to sort our DataFrame not only to view it but also to do operations as in this case:

In [10]:
out = (df.sort("Fare", descending=True).group_by("Pclass").agg(
        pl.col('Name').first().alias("richer"),
        pl.col('Name').last().alias("poorer")).limit(3))
print(out)

shape: (3, 3)
┌────────┬────────────────────────────┬─────────────────────────────────┐
│ Pclass ┆ richer                     ┆ poorer                          │
│ ---    ┆ ---                        ┆ ---                             │
│ i64    ┆ str                        ┆ str                             │
╞════════╪════════════════════════════╪═════════════════════════════════╡
│ 1      ┆ Ward, Miss. Anna           ┆ Reuchlin, Jonkheer. John George │
│ 3      ┆ Sage, Master. Thomas Henry ┆ Johnson, Mr. Alfred             │
│ 2      ┆ Hood, Mr. Ambrose Jr       ┆ Knight, Mr. Robert J            │
└────────┴────────────────────────────┴─────────────────────────────────┘


### Expressions

We have so many expressions in polar that for the sake of brevity I will list only the most common ones, and I encourage you to consult the library guide for the complete list.

##### Numerical and Logical

Clearly the four operations and the logical relationships are supported:

In [11]:
out = df.select(
        (pl.col('Fare') * 100).alias('¢'),
        ((pl.col("Age") <= 10) & (pl.col("Survived") == 1.)).alias("Is a child survivor?")).limit(5)
print(out)

shape: (5, 2)
┌─────────┬──────────────────────┐
│ ¢       ┆ Is a child survivor? │
│ ---     ┆ ---                  │
│ f64     ┆ bool                 │
╞═════════╪══════════════════════╡
│ 725.0   ┆ false                │
│ 7128.33 ┆ false                │
│ 792.5   ┆ false                │
│ 5310.0  ┆ false                │
│ 805.0   ┆ false                │
└─────────┴──────────────────────┘


##### Expression expansion

At first glance this section might seem almost superfluous or otherwise simple and unimportant, but mastering "Expression expansion" can significantly reduce the code you will have to write.

In [12]:
out = df.select(pl.col("*")).limit(3) # equivalent to df.select(pl.all())
print(out)

shape: (3, 12)
┌─────────────┬──────────┬────────┬───────────────────┬───┬───────────┬─────────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name              ┆ … ┆ Ticket    ┆ Fare    ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---               ┆   ┆ ---       ┆ ---     ┆ ---   ┆ ---      │
│ i64         ┆ i64      ┆ i64    ┆ str               ┆   ┆ str       ┆ f64     ┆ str   ┆ str      │
╞═════════════╪══════════╪════════╪═══════════════════╪═══╪═══════════╪═════════╪═══════╪══════════╡
│ 1           ┆ 0        ┆ 3      ┆ Braund, Mr. Owen  ┆ … ┆ A/5 21171 ┆ 7.25    ┆ null  ┆ S        │
│             ┆          ┆        ┆ Harris            ┆   ┆           ┆         ┆       ┆          │
│ 2           ┆ 1        ┆ 1      ┆ Cumings, Mrs.     ┆ … ┆ PC 17599  ┆ 71.2833 ┆ C85   ┆ C        │
│             ┆          ┆        ┆ John Bradley      ┆   ┆           ┆         ┆       ┆          │
│             ┆          ┆        ┆ (Flor…            ┆   ┆           ┆     

we can select one or more columns by simply typing its name in the expression ```pl.col```, or we can exclude from visualization with:  

In [13]:
out = df.select(pl.all().exclude("Ticket","Embarked","Name","Cabin","Parch")).limit(3)
print(out)

shape: (3, 7)
┌─────────────┬──────────┬────────┬────────┬──────┬───────┬─────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Sex    ┆ Age  ┆ SibSp ┆ Fare    │
│ ---         ┆ ---      ┆ ---    ┆ ---    ┆ ---  ┆ ---   ┆ ---     │
│ i64         ┆ i64      ┆ i64    ┆ str    ┆ f64  ┆ i64   ┆ f64     │
╞═════════════╪══════════╪════════╪════════╪══════╪═══════╪═════════╡
│ 1           ┆ 0        ┆ 3      ┆ male   ┆ 22.0 ┆ 1     ┆ 7.25    │
│ 2           ┆ 1        ┆ 1      ┆ female ┆ 38.0 ┆ 1     ┆ 71.2833 │
│ 3           ┆ 1        ┆ 3      ┆ female ┆ 26.0 ┆ 0     ┆ 7.925   │
└─────────────┴──────────┴────────┴────────┴──────┴───────┴─────────┘


You can also select columns by type, regular expressions, or using selectors, which I invite you to view on your own

##### Column naming

Perform an operation on a column does not change its name if we want to do it we have to use the ```.alias()``` function as we have partly already seen:

In [14]:
out = df.select((pl.col('Fare') * 100).alias('¢'),
          (pl.col('Fare') / 1000).alias('k$')).limit(3)
print(out)

shape: (3, 2)
┌─────────┬───────────┐
│ ¢       ┆ k$        │
│ ---     ┆ ---       │
│ f64     ┆ f64       │
╞═════════╪═══════════╡
│ 725.0   ┆ 0.00725   │
│ 7128.33 ┆ 0.0712833 │
│ 792.5   ┆ 0.007925  │
└─────────┴───────────┘


##### Count unique values

Unique values can be counted with two algorithms, one exact and one approximated with HyperLogLog++. The advantage of approximation occurs when we have many rows:

In [15]:
out = df.select(
    pl.col("Name").n_unique().alias("unique"),
    pl.approx_n_unique("Name").alias("unique_approx"))
print(out)

shape: (1, 2)
┌────────┬───────────────┐
│ unique ┆ unique_approx │
│ ---    ┆ ---           │
│ u32    ┆ u32           │
╞════════╪═══════════════╡
│ 891    ┆ 891           │
└────────┴───────────────┘


##### Conditionals

Polars supports if-else like conditions in expressions:

In [16]:
out = df.select(
    pl.col("Age"),
    pl.when(pl.col("Age") > 18)
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .alias("conditional")).limit(5)
print(out)

shape: (5, 2)
┌──────┬─────────────┐
│ Age  ┆ conditional │
│ ---  ┆ ---         │
│ f64  ┆ bool        │
╞══════╪═════════════╡
│ 22.0 ┆ true        │
│ 38.0 ┆ true        │
│ 26.0 ┆ true        │
│ 35.0 ┆ true        │
│ 35.0 ┆ true        │
└──────┴─────────────┘


The predicate is placed in the when clause, and if its evaluation is true, the then expression is applied, otherwise the otherwise expression is applied

##### Casting

Polars uses Arrow to manage the data in memory and relies on Rust to do the conversion

In [17]:
df.limit(2)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""


In [18]:
out = df.select(pl.col('Fare').cast(pl.Int32), #Note that in the case of decimal values these are rounded downwards when casting to an integer.
                pl.col('Pclass').cast(pl.String).cast(pl.Float32),
                pl.col('Survived').cast(pl.Boolean)).limit(3)
print(out)

shape: (3, 3)
┌──────┬────────┬──────────┐
│ Fare ┆ Pclass ┆ Survived │
│ ---  ┆ ---    ┆ ---      │
│ i32  ┆ f32    ┆ bool     │
╞══════╪════════╪══════════╡
│ 7    ┆ 3.0    ┆ false    │
│ 71   ┆ 1.0    ┆ true     │
│ 7    ┆ 3.0    ┆ true     │
└──────┴────────┴──────────┘


The cast method includes a strict parameter that determines the behavior of Polars when it encounters a value that cannot be converted from the source DataType to the target DataType. By default, strict=True means that Polars will throw an error to notify the user of the failed conversion and provide details about the values that could not be cast. If strict=False instead, values that cannot be converted to the target DataType will be safely converted to null.

It is also possible to convert dates to string and viceversa:

In [64]:
from datetime import date, datetime

df_dates = pl.DataFrame(
    {
        "date": pl.date_range(date(2020, 2, 19), date(2024, 2, 23), interval='1y', eager=True),
        "string": ["2020-02-19","2021-02-19","2022-02-19","2023-02-19","2024-02-23",],
        "values": [12,3,4,5,7],
        "groups": ['a','b','b','c','a']
    }
)

out = df_dates.select(
    pl.col("date").dt.to_string("%Y-%m-%d"),
    pl.col("string").str.to_datetime("%Y-%m-%d"),
)
print(out)


shape: (5, 2)
┌────────────┬─────────────────────┐
│ date       ┆ string              │
│ ---        ┆ ---                 │
│ str        ┆ datetime[μs]        │
╞════════════╪═════════════════════╡
│ 2020-02-19 ┆ 2020-02-19 00:00:00 │
│ 2021-02-19 ┆ 2021-02-19 00:00:00 │
│ 2022-02-19 ┆ 2022-02-19 00:00:00 │
│ 2023-02-19 ┆ 2023-02-19 00:00:00 │
│ 2024-02-19 ┆ 2024-02-23 00:00:00 │
└────────────┴─────────────────────┘


##### Strings

String processing can often be inefficient because of their unpredictable memory size, which forces the CPU to access many random memory locations. To solve this problem, Polars uses Arrow as a backend, which stores all strings in a contiguous block of memory. The string namespace can be accessed through the ```.str``` attribute and then we can perform our operation:

In [65]:
out = df.select(
    pl.col("Name"),
    pl.col("Name").str.contains("Mr|Mrs").alias("regex"),
    pl.col("Name").str.contains("Miss$", literal=True).alias("literal"),
    pl.col("Name").str.starts_with("A").alias("starts_with"),
    pl.col("Name").str.ends_with("Z").alias("ends_with")).limit(3)
print(out)

shape: (3, 5)
┌───────────────────────────────────┬───────┬─────────┬─────────────┬───────────┐
│ Name                              ┆ regex ┆ literal ┆ starts_with ┆ ends_with │
│ ---                               ┆ ---   ┆ ---     ┆ ---         ┆ ---       │
│ str                               ┆ bool  ┆ bool    ┆ bool        ┆ bool      │
╞═══════════════════════════════════╪═══════╪═════════╪═════════════╪═══════════╡
│ Braund, Mr. Owen Harris           ┆ true  ┆ false   ┆ false       ┆ false     │
│ Cumings, Mrs. John Bradley (Flor… ┆ true  ┆ false   ┆ false       ┆ false     │
│ Heikkinen, Miss. Laina            ┆ false ┆ false   ┆ false       ┆ false     │
└───────────────────────────────────┴───────┴─────────┴─────────────┴───────────┘


we can also extract a pattern:

In [66]:
out = df.select(pl.col("Name").str.extract(r"\.\s*(\w+)", group_index=1)).limit(3)
print(out)

shape: (3, 1)
┌───────┐
│ Name  │
│ ---   │
│ str   │
╞═══════╡
│ Owen  │
│ John  │
│ Laina │
└───────┘


this will provide only the 'group_index' occurrence, if we want all occurrences we have to use ```str.extract_all()```.
We can even do some replace:

In [67]:
out = df.select(pl.col("Name").str.replace(r"(Mrs|Miss|Mr)", "Mx")).limit(3)
print(out)

shape: (3, 1)
┌───────────────────────────────────┐
│ Name                              │
│ ---                               │
│ str                               │
╞═══════════════════════════════════╡
│ Braund, Mx. Owen Harris           │
│ Cumings, Mx. John Bradley (Flore… │
│ Heikkinen, Mx. Laina              │
└───────────────────────────────────┘


##### Missing data

Missing data are represented in Arrow and Polars with a ```pl.Null``` value. This null missing value applies to all data types, including numeric values

In [68]:
out = df.null_count()
print(out)

shape: (1, 12)
┌─────────────┬──────────┬────────┬──────┬───┬────────┬──────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name ┆ … ┆ Ticket ┆ Fare ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---  ┆   ┆ ---    ┆ ---  ┆ ---   ┆ ---      │
│ u32         ┆ u32      ┆ u32    ┆ u32  ┆   ┆ u32    ┆ u32  ┆ u32   ┆ u32      │
╞═════════════╪══════════╪════════╪══════╪═══╪════════╪══════╪═══════╪══════════╡
│ 0           ┆ 0        ┆ 0      ┆ 0    ┆ … ┆ 0      ┆ 0    ┆ 687   ┆ 2        │
└─────────────┴──────────┴────────┴──────┴───┴────────┴──────┴───────┴──────────┘


To replace null values we can use:

In [69]:
out = df.with_columns(pl.col("Cabin").fill_null(pl.lit('No cabin'))).limit(3)
print(out)

shape: (3, 12)
┌─────────────┬──────────┬────────┬──────────────┬───┬─────────────┬─────────┬──────────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name         ┆ … ┆ Ticket      ┆ Fare    ┆ Cabin    ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---          ┆   ┆ ---         ┆ ---     ┆ ---      ┆ ---      │
│ i64         ┆ i64      ┆ i64    ┆ str          ┆   ┆ str         ┆ f64     ┆ str      ┆ str      │
╞═════════════╪══════════╪════════╪══════════════╪═══╪═════════════╪═════════╪══════════╪══════════╡
│ 1           ┆ 0        ┆ 3      ┆ Braund, Mr.  ┆ … ┆ A/5 21171   ┆ 7.25    ┆ No cabin ┆ S        │
│             ┆          ┆        ┆ Owen Harris  ┆   ┆             ┆         ┆          ┆          │
│ 2           ┆ 1        ┆ 1      ┆ Cumings,     ┆ … ┆ PC 17599    ┆ 71.2833 ┆ C85      ┆ C        │
│             ┆          ┆        ┆ Mrs. John    ┆   ┆             ┆         ┆          ┆          │
│             ┆          ┆        ┆ Bradley      ┆   ┆             ┆        

when we use the ```fill_null``` function we can also establish a strategy as in pandas

##### Over

The over function allows operations per group useful for example in creating new features:

In [70]:
out = df.with_columns(pl.col("Fare").mean().over("Pclass").alias('avg Fare per class')).limit(3)
print(out)

shape: (3, 13)
┌─────────────┬──────────┬────────┬───────────────┬───┬─────────┬───────┬──────────┬───────────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name          ┆ … ┆ Fare    ┆ Cabin ┆ Embarked ┆ avg Fare per  │
│ ---         ┆ ---      ┆ ---    ┆ ---           ┆   ┆ ---     ┆ ---   ┆ ---      ┆ class         │
│ i64         ┆ i64      ┆ i64    ┆ str           ┆   ┆ f64     ┆ str   ┆ str      ┆ ---           │
│             ┆          ┆        ┆               ┆   ┆         ┆       ┆          ┆ f64           │
╞═════════════╪══════════╪════════╪═══════════════╪═══╪═════════╪═══════╪══════════╪═══════════════╡
│ 1           ┆ 0        ┆ 3      ┆ Braund, Mr.   ┆ … ┆ 7.25    ┆ null  ┆ S        ┆ 13.67555      │
│             ┆          ┆        ┆ Owen Harris   ┆   ┆         ┆       ┆          ┆               │
│ 2           ┆ 1        ┆ 1      ┆ Cumings, Mrs. ┆ … ┆ 71.2833 ┆ C85   ┆ C        ┆ 84.154687     │
│             ┆          ┆        ┆ John Bradley  ┆   ┆         ┆       ┆   

let's verify it:

In [71]:
out = df.select(pl.col('Fare','Pclass')).group_by('Pclass').mean()
print(out)

shape: (3, 2)
┌────────┬───────────┐
│ Pclass ┆ Fare      │
│ ---    ┆ ---       │
│ i64    ┆ f64       │
╞════════╪═══════════╡
│ 3      ┆ 13.67555  │
│ 1      ┆ 84.154687 │
│ 2      ┆ 20.662183 │
└────────┴───────────┘


##### Fold

Folds are used for accumulating on multiple columns horizontally and can be very useful:

In [72]:
df_fold = pl.DataFrame({"a": [1, 2, 3],"b": [4, 5, 1]})

out = df_fold.select(
    pl.fold(acc=pl.lit(0), function=lambda acc, x: acc + x, exprs=pl.all()>1).alias("count if > 1"))
print(out)

shape: (3, 1)
┌──────────────┐
│ count if > 1 │
│ ---          │
│ i32          │
╞══════════════╡
│ 1            │
│ 2            │
│ 1            │
└──────────────┘


### Combining DataFrames

there are two ways to combine DataFrames in polars, methods that we also seen in pandas but first we create two dataframes to use then:

In [73]:
import random
import string

df1 = pl.DataFrame(
    {
        "index": range(8),
        "random numbers": [random.random() for _ in range(8)],
        "x": [1, 2.0, float("nan"), float("nan"), 0, -5, -42, None],
    }
)

df2 = pl.DataFrame(
    {
        "id": range(8),
        "random letters": [random.choice(string.ascii_letters) for _ in range(8)],
    }
)

df3 = pl.DataFrame(
    {
        "index": 8,
        "random numbers": random.random(),
        "x": 20.0,
        

    }
)

##### Join

The join is a cotuct borrowed from the sql language different types of joins are implemented for their explanation I refer to the literature:

In [74]:
joined = df1.join(df2, left_on="index", right_on="id")
print(joined)

shape: (8, 4)
┌───────┬────────────────┬───────┬────────────────┐
│ index ┆ random numbers ┆ x     ┆ random letters │
│ ---   ┆ ---            ┆ ---   ┆ ---            │
│ i64   ┆ f64            ┆ f64   ┆ str            │
╞═══════╪════════════════╪═══════╪════════════════╡
│ 0     ┆ 0.066844       ┆ 1.0   ┆ O              │
│ 1     ┆ 0.589281       ┆ 2.0   ┆ b              │
│ 2     ┆ 0.417543       ┆ NaN   ┆ V              │
│ 3     ┆ 0.164582       ┆ NaN   ┆ e              │
│ 4     ┆ 0.814405       ┆ 0.0   ┆ C              │
│ 5     ┆ 0.676169       ┆ -5.0  ┆ E              │
│ 6     ┆ 0.644668       ┆ -42.0 ┆ e              │
│ 7     ┆ 0.894817       ┆ null  ┆ a              │
└───────┴────────────────┴───────┴────────────────┘


##### Concat

Concat is short for concatenate; you can concatenate both vertically and horizontally. Of course, one must be careful about size when using this construct:

In [75]:
hstacked = df1.hstack(df2)
print(hstacked)

shape: (8, 5)
┌───────┬────────────────┬───────┬─────┬────────────────┐
│ index ┆ random numbers ┆ x     ┆ id  ┆ random letters │
│ ---   ┆ ---            ┆ ---   ┆ --- ┆ ---            │
│ i64   ┆ f64            ┆ f64   ┆ i64 ┆ str            │
╞═══════╪════════════════╪═══════╪═════╪════════════════╡
│ 0     ┆ 0.066844       ┆ 1.0   ┆ 0   ┆ O              │
│ 1     ┆ 0.589281       ┆ 2.0   ┆ 1   ┆ b              │
│ 2     ┆ 0.417543       ┆ NaN   ┆ 2   ┆ V              │
│ 3     ┆ 0.164582       ┆ NaN   ┆ 3   ┆ e              │
│ 4     ┆ 0.814405       ┆ 0.0   ┆ 4   ┆ C              │
│ 5     ┆ 0.676169       ┆ -5.0  ┆ 5   ┆ E              │
│ 6     ┆ 0.644668       ┆ -42.0 ┆ 6   ┆ e              │
│ 7     ┆ 0.894817       ┆ null  ┆ 7   ┆ a              │
└───────┴────────────────┴───────┴─────┴────────────────┘


when using hstack also pay attention to column names:

In [76]:
vstacked = df1.vstack(df3)
print(vstacked)

shape: (9, 3)
┌───────┬────────────────┬───────┐
│ index ┆ random numbers ┆ x     │
│ ---   ┆ ---            ┆ ---   │
│ i64   ┆ f64            ┆ f64   │
╞═══════╪════════════════╪═══════╡
│ 0     ┆ 0.066844       ┆ 1.0   │
│ 1     ┆ 0.589281       ┆ 2.0   │
│ 2     ┆ 0.417543       ┆ NaN   │
│ 3     ┆ 0.164582       ┆ NaN   │
│ 4     ┆ 0.814405       ┆ 0.0   │
│ 5     ┆ 0.676169       ┆ -5.0  │
│ 6     ┆ 0.644668       ┆ -42.0 │
│ 7     ┆ 0.894817       ┆ null  │
│ 8     ┆ 0.913987       ┆ 20.0  │
└───────┴────────────────┴───────┘


##### Pivots and Melts

Pivots are operations that transform a column of a DataFrame into a new dimension by aggregating values according to a chosen function. In contrast, melts are the inverse operations that deconstruct a DataFrame

In [77]:
out = df.melt(id_vars=['PassengerId'], value_vars=['Fare','Survived'])
print(out)

shape: (1_782, 3)
┌─────────────┬──────────┬─────────┐
│ PassengerId ┆ variable ┆ value   │
│ ---         ┆ ---      ┆ ---     │
│ i64         ┆ str      ┆ f64     │
╞═════════════╪══════════╪═════════╡
│ 1           ┆ Fare     ┆ 7.25    │
│ 2           ┆ Fare     ┆ 71.2833 │
│ 3           ┆ Fare     ┆ 7.925   │
│ 4           ┆ Fare     ┆ 53.1    │
│ …           ┆ …        ┆ …       │
│ 888         ┆ Survived ┆ 1.0     │
│ 889         ┆ Survived ┆ 0.0     │
│ 890         ┆ Survived ┆ 1.0     │
│ 891         ┆ Survived ┆ 0.0     │
└─────────────┴──────────┴─────────┘


contra:

In [78]:
out = out.pivot(index=['PassengerId'], columns='variable', values='value')
print(out)

shape: (891, 3)
┌─────────────┬─────────┬──────────┐
│ PassengerId ┆ Fare    ┆ Survived │
│ ---         ┆ ---     ┆ ---      │
│ i64         ┆ f64     ┆ f64      │
╞═════════════╪═════════╪══════════╡
│ 1           ┆ 7.25    ┆ 0.0      │
│ 2           ┆ 71.2833 ┆ 1.0      │
│ 3           ┆ 7.925   ┆ 1.0      │
│ 4           ┆ 53.1    ┆ 1.0      │
│ …           ┆ …       ┆ …        │
│ 888         ┆ 30.0    ┆ 1.0      │
│ 889         ┆ 23.45   ┆ 0.0      │
│ 890         ┆ 30.0    ┆ 1.0      │
│ 891         ┆ 7.75    ┆ 0.0      │
└─────────────┴─────────┴──────────┘


### Time series

Working with time series, it is important to code dates in the correct manner because polars has native support for analyzing time series data to perform more sophisticated operations such as time grouping and resampling.

Start to cast in the right format:

In [79]:
df_dates = df_dates.with_columns(pl.col("date").cast(pl.Datetime).alias('datetime'))
print(df_dates)

shape: (5, 5)
┌────────────┬────────────┬────────┬────────┬─────────────────────┐
│ date       ┆ string     ┆ values ┆ groups ┆ datetime            │
│ ---        ┆ ---        ┆ ---    ┆ ---    ┆ ---                 │
│ date       ┆ str        ┆ i64    ┆ str    ┆ datetime[μs]        │
╞════════════╪════════════╪════════╪════════╪═════════════════════╡
│ 2020-02-19 ┆ 2020-02-19 ┆ 12     ┆ a      ┆ 2020-02-19 00:00:00 │
│ 2021-02-19 ┆ 2021-02-19 ┆ 3      ┆ b      ┆ 2021-02-19 00:00:00 │
│ 2022-02-19 ┆ 2022-02-19 ┆ 4      ┆ b      ┆ 2022-02-19 00:00:00 │
│ 2023-02-19 ┆ 2023-02-19 ┆ 5      ┆ c      ┆ 2023-02-19 00:00:00 │
│ 2024-02-19 ┆ 2024-02-23 ┆ 7      ┆ a      ┆ 2024-02-19 00:00:00 │
└────────────┴────────────┴────────┴────────┴─────────────────────┘


We can extract date features from a date column:

In [80]:
out = df_dates.with_columns(pl.col("date").dt.year().alias("year"))
print(out)

shape: (5, 6)
┌────────────┬────────────┬────────┬────────┬─────────────────────┬──────┐
│ date       ┆ string     ┆ values ┆ groups ┆ datetime            ┆ year │
│ ---        ┆ ---        ┆ ---    ┆ ---    ┆ ---                 ┆ ---  │
│ date       ┆ str        ┆ i64    ┆ str    ┆ datetime[μs]        ┆ i32  │
╞════════════╪════════════╪════════╪════════╪═════════════════════╪══════╡
│ 2020-02-19 ┆ 2020-02-19 ┆ 12     ┆ a      ┆ 2020-02-19 00:00:00 ┆ 2020 │
│ 2021-02-19 ┆ 2021-02-19 ┆ 3      ┆ b      ┆ 2021-02-19 00:00:00 ┆ 2021 │
│ 2022-02-19 ┆ 2022-02-19 ┆ 4      ┆ b      ┆ 2022-02-19 00:00:00 ┆ 2022 │
│ 2023-02-19 ┆ 2023-02-19 ┆ 5      ┆ c      ┆ 2023-02-19 00:00:00 ┆ 2023 │
│ 2024-02-19 ┆ 2024-02-23 ┆ 7      ┆ a      ┆ 2024-02-19 00:00:00 ┆ 2024 │
└────────────┴────────────┴────────┴────────┴─────────────────────┴──────┘


Working with time zones:

In [81]:
out = df_dates.select(pl.col('datetime').dt.replace_time_zone("UTC").dt.convert_time_zone("Europe/Brussels"))
print(out)

shape: (5, 1)
┌───────────────────────────────┐
│ datetime                      │
│ ---                           │
│ datetime[μs, Europe/Brussels] │
╞═══════════════════════════════╡
│ 2020-02-19 01:00:00 CET       │
│ 2021-02-19 01:00:00 CET       │
│ 2022-02-19 01:00:00 CET       │
│ 2023-02-19 01:00:00 CET       │
│ 2024-02-19 01:00:00 CET       │
└───────────────────────────────┘


Filter in a better way:

In [82]:
out = df_dates.filter(pl.col("datetime") == datetime(2020, 2, 19))
print(out)

shape: (1, 5)
┌────────────┬────────────┬────────┬────────┬─────────────────────┐
│ date       ┆ string     ┆ values ┆ groups ┆ datetime            │
│ ---        ┆ ---        ┆ ---    ┆ ---    ┆ ---                 │
│ date       ┆ str        ┆ i64    ┆ str    ┆ datetime[μs]        │
╞════════════╪════════════╪════════╪════════╪═════════════════════╡
│ 2020-02-19 ┆ 2020-02-19 ┆ 12     ┆ a      ┆ 2020-02-19 00:00:00 │
└────────────┴────────────┴────────┴────────┴─────────────────────┘


or even:

In [83]:
out = df_dates.filter(pl.col("datetime").is_between(datetime(2020, 1, 1), datetime(2022, 1, 1)))
print(out)

shape: (2, 5)
┌────────────┬────────────┬────────┬────────┬─────────────────────┐
│ date       ┆ string     ┆ values ┆ groups ┆ datetime            │
│ ---        ┆ ---        ┆ ---    ┆ ---    ┆ ---                 │
│ date       ┆ str        ┆ i64    ┆ str    ┆ datetime[μs]        │
╞════════════╪════════════╪════════╪════════╪═════════════════════╡
│ 2020-02-19 ┆ 2020-02-19 ┆ 12     ┆ a      ┆ 2020-02-19 00:00:00 │
│ 2021-02-19 ┆ 2021-02-19 ┆ 3      ┆ b      ┆ 2021-02-19 00:00:00 │
└────────────┴────────────┴────────┴────────┴─────────────────────┘


We can even have a very nice function for grouping:

In [85]:
out = df_dates.group_by_dynamic("datetime", every="2y", period="3y", by="groups").agg(pl.col("values").mean())
print(out)

shape: (7, 3)
┌────────┬─────────────────────┬────────┐
│ groups ┆ datetime            ┆ values │
│ ---    ┆ ---                 ┆ ---    │
│ str    ┆ datetime[μs]        ┆ f64    │
╞════════╪═════════════════════╪════════╡
│ a      ┆ 2018-01-01 00:00:00 ┆ 12.0   │
│ a      ┆ 2020-01-01 00:00:00 ┆ 12.0   │
│ a      ┆ 2022-01-01 00:00:00 ┆ 7.0    │
│ a      ┆ 2024-01-01 00:00:00 ┆ 7.0    │
│ b      ┆ 2020-01-01 00:00:00 ┆ 3.5    │
│ b      ┆ 2022-01-01 00:00:00 ┆ 4.0    │
│ c      ┆ 2022-01-01 00:00:00 ┆ 5.0    │
└────────┴─────────────────────┴────────┘


Lastly we can even downsample the time series:

In [96]:
out = df_dates.upsample(time_column="datetime", every="6mo").fill_null(strategy="forward").limit(5)
print(out)

shape: (5, 5)
┌─────────────────────┬────────────┬────────────┬────────┬────────┐
│ datetime            ┆ date       ┆ string     ┆ values ┆ groups │
│ ---                 ┆ ---        ┆ ---        ┆ ---    ┆ ---    │
│ datetime[μs]        ┆ date       ┆ str        ┆ i64    ┆ str    │
╞═════════════════════╪════════════╪════════════╪════════╪════════╡
│ 2020-02-19 00:00:00 ┆ 2020-02-19 ┆ 2020-02-19 ┆ 12     ┆ a      │
│ 2020-08-19 00:00:00 ┆ 2020-02-19 ┆ 2020-02-19 ┆ 12     ┆ a      │
│ 2021-02-19 00:00:00 ┆ 2021-02-19 ┆ 2021-02-19 ┆ 3      ┆ b      │
│ 2021-08-19 00:00:00 ┆ 2021-02-19 ┆ 2021-02-19 ┆ 3      ┆ b      │
│ 2022-02-19 00:00:00 ┆ 2022-02-19 ┆ 2022-02-19 ┆ 4      ┆ b      │
└─────────────────────┴────────────┴────────────┴────────┴────────┘


What about downsampling? Think of it as homework!

### Lazy